In [1]:
import xarray as xr
import glob

from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster

# ERA5 Land data: from NetCDF3 to Zarr

In this notebook we load ERA5 Land data stored in a collection of NetCDF3 files and we save it as a (chunked) Zarr store. It is global.

## Input variables

In [15]:
year = 2015

In [16]:
ROOT_DIR = '/gpfs/work2/0/ttse0619'
ERA5_DIR = (
    f'{ROOT_DIR}/qianqian/global_data_Qianqian/'
    f'1input_data/{year}global/era5land/'
)
OUT_DIR = (
    f'{ROOT_DIR}/francesco/Projects/EcoExtreML/Data/'
    f'1input_data/{year}global/era5land'
)

## Setup Dask cluster

NOTE: when working with NetCDF files (and the netcdf4 library) it is much better to work with many processes and few threads per process: netcdf4 can only read from one thread per process.

In [3]:
# cluster = LocalCluster(n_workers=16, threads_per_worker=1)

In [5]:
cluster = SLURMCluster(
    name='dask-worker',
    cores=16,
    processes=16,
    queue='fat',
    memory='120GiB',
    local_directory='$TMPDIR',
    walltime='5:00:00'
)

We use in total 4 * 16 = 64 single-threaded workers, and ~480 GiB total memory: 

In [9]:
cluster.scale(jobs=4)

In [10]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://145.136.56.49:33215,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


## Converting the dataset 

In [18]:
era5_paths = sorted(
    glob.glob(f'{ERA5_DIR}/era5-land_*.nc')
)

In [19]:
ds = xr.open_mfdataset(era5_paths, chunks={'longitude': 250, 'latitude': 250})
ds = ds.chunk({'time': 750})

The resulting dataset has 8 variables, with chunks of approximately 180 MiB:

In [22]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 8760)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 23:48:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [24]:
%%time
zarr_path = f'{OUT_DIR}/era5land.zarr'
ds.to_zarr(zarr_path, mode='w')

CPU times: user 6min 45s, sys: 18.8 s, total: 7min 4s
Wall time: 1h 12min 54s


When done with the conversion we switch off the cluster to release resources:

In [25]:
client.shutdown()

The overall Zarr archive is ~200 GB:

In [29]:
!du -h $zarr_path

31G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/sp
18G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/ssrd
30G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/v10
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/longitude
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/latitude
30G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/d2m
11G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/tp
29G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5land/era5land.zarr/u10
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/1input_data/2015global/era5la